In [ ]:
import spacy
import pytest
from bedoner.entity_extractors.bert_ner import BertEntityExtractor, create_estimator
from pathlib import Path
import pickle
from spacy.tokens import Doc
from bedoner.lang.juman import Japanese as Juman
from spacy.strings import StringStore
from spacy.vocab import Vocab
from spacy_pytorch_transformers.pipeline.wordpiecer import PyTT_WordPiecer
from bedoner.lang.mecab import Japanese
from bedoner.wordpiecer import BertWordPiecer
import json
from pathlib import Path 
import shutil
from spacy.cli import package
import tempfile

In [ ]:
name="bert_ner"

In [ ]:
def create_package(nlp):
    meta=nlp.meta
    req=meta.get("requirements") or []
    req.append("bedoner @ git+https://github.com/PKSHATechnology/bedore-ner")
    nlp.meta["requirements"] = req
    
    pkgs = Path("../pkgs")
    with tempfile.TemporaryDirectory() as tmpd:
        nlp.to_disk(str(tmpd))
        package(tmpd, pkgs, force=True)
    model_name  =  meta["lang"] + "_" + meta["name"]
    pkgd = pkgs / (model_name+ "-" + meta["version"])
    return pkgd, tmpd

In [ ]:
__dir__ = Path(".").parent

In [ ]:
vocabfile = __dir__ / "../data/Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt"
with vocabfile.open() as f:
    vs = []
    for line in f:
        vs.append(line[:-1])
s = StringStore(vs)
v = Vocab(strings=s)
nlp = Juman(v, meta={"name": name})
w = BertWordPiecer(
    v,
    vocab_file=str(vocabfile)
)
w.model = w.Model(w.cfg["vocab_file"])
nlp.add_pipe(w)

In [ ]:
bert_dir = __dir__ / "../data/Japanese_L-12_H-768_A-12_E-30_BPE"
model_dir = __dir__ / "../data/bert_result_ene_0/"
init_checkpoint = str(bert_dir / "bert_model.ckpt")
with (model_dir / "label2id.json").open("r") as f:
    label2id = json.load(f)

bert_cfg = dict(
    bert_dir=str(bert_dir),
    model_dir=str(model_dir),
    num_labels=len(label2id) + 1,
    init_checkpoint=init_checkpoint,
    use_one_hot_embeddings=None,
    max_seq_length=128,
    batch_size=10,
)

ee = BertEntityExtractor.from_nlp(nlp, label2id=label2id, **bert_cfg)
ee.model = create_estimator(**bert_cfg)
ee.set_values()
ee.create_predictor()
nlp.add_pipe(ee)

In [ ]:
doc=nlp("EXILEのATSUSHIと中島美嘉が14日ニューヨーク入り") 
doc.ents

In [ ]:
nlp.to_disk("foo")

In [ ]:
pkgd, tmpd = create_package(nlp)